# Weather Agent

This Agent:

- Has a **tool** to **fetch mock weather data**
- Has no LLM
- Implements the `handle_task` function.

The agent requires a **transport** to send and receive messages and more specifically an **AgentTransport**. In the example below we use the **HTTPAgentTransport**, which uses an HTTP Client and Server (REST API) for **Agent-to-Agent** communication. Also for **Agent-to-Registry** communication we'll use the HTTP Registry Trasport.

**Note**: The Agent accepts a **Registry**, **RegistryClient** or **string** as a `registry` argument. If a string is provided, it is expected to be a URL to a Registry API endpoint for an HTTPRegistry.

In [ ]:
from protolink.agents import Agent
from protolink.models import AgentCard, Task
from protolink.transport import HTTPAgentTransport

# Weather Agent URL
URL = "http://localhost:8010"

# This could be an env variable
REGISTRY_URL = "http://localhost:9000"

# A2A Transport
transport = HTTPAgentTransport(url=URL)

### Define the Agent Class

The agent inherits from the `Agent` class and implements the `handle_task` method, which is **mandatory** to implement if the Agent is expected to handle tasks from other Agents. 

In [ ]:
class WeatherAgent(Agent):
    async def handle_task(self, task: Task):
        result = await self.call_tool("get_weather", city="Geneva")
        return task.complete(f"Weather data: {result}")

### Create Agent object

When Initiating an Agent object the `card` argument is mandatory. The `card` argument is an instance of the `AgentCard` class, which is used to define the agent's identity data. The mandatory fields are `name`, `description`, and `url`.

The Agent also accepts a `dict` isntead of an `AgentCard` instance. This is useful for when you want to define the agent's identity data in a more dynamic way.

In [ ]:
# Define Agent Card using the Agent Card class
card = AgentCard(url=URL, name="WeatherAgent", description="Produces weather data")

# Initialize the agent object
agent = WeatherAgent(card=card, transport=transport, registry=REGISTRY_URL)

### Add native Tool to Agent

Using the **`@tool`** decorator we can add a native tool to the agent. The `name` and `description` arguments are mandatory. See tool base class to see what other arguments are available.

In [ ]:
# Add Native tool using the decorator
@agent.tool(name="get_weather", description="Return weather data for a city")
async def get_weather(city: str):
    # tool logic here
    return {"city": city, "temperature": 28, "condition": "sunny"}

### Start the Agent

When `start()` is called the agent will:

- Register with the Registry (if `register` is True). Initiates the agent's **RegistryClient** which uses the transport to send messages, for Agent-to-Registry Communication.
- Start the **AgentServer** which uses the transport to receive messages, for Agent-to-Agent Communication

- **POST** `/tasks`: Sends a task to another agent.
- **GET** `/.well-known/agent.json` or `/`: Returns the agent card.
- **GET** `/status`: Returns the status of the agent. Then by accessing in your **browser** `{agent_url}/status` (i.e. http://localhost:8010/status) an HTML page with the **status** of the agent will be displayed.

In [ ]:
await agent.start(register=True)